# Do photometry of 3I and (22281) Popescu

- Results are saved as `photres_3I_mean_600s_20250715.txt` and `photres_ref_mean_600s_20250715.txt` (for 3I)
- Results are saved as `photres_P_mean_120s_20250715.txt` and `photres_ref_mean_120s_20250715.txt` (for Popescu)

In [7]:
import os 
from argparse import ArgumentParser as ap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
%matplotlib inline
from astropy.io import fits as fits
import sep
from matplotlib.patches import Circle
from astroquery.jplhorizons import Horizons
from scipy.ndimage import gaussian_filter
from astropy.wcs import WCS as wcs
from astropy.coordinates import SkyCoord
from astropy import units as u
from astroquery.mast import Catalogs


# Aperture radii in pix
# Note: seeing ~ 2.2–2.6 arcsec (6-8 pix)
rad_list = [5, 6, 7, 8, 9, 10, 11, 12]

# Size of FoV
fov_arcsec = 30

# Stacking 
stack = "mean"

# Dynamic range of the figures
sig_min = -0.5
sig_max = 5

# Output directory
outdir = "../data/"

In [8]:
def query_ps(ra, dec, radius, magmin, magmax):
  """
  Query Pan-STARES catalog and output pandas.Data.
  Extract bright and high quality objects.

  Parameters
  ----------
  ra, dec : float
    right ascension, declination of field in unit of degree.
  radius : int
    radius of filed of view in unit of degree
  magmin, magmax : float
    the brightest/faintest magnitude to be considered (g-band)

  Return
  ------
  df : pandas.DataFrame
    result DataFrame
  """
  radec = SkyCoord(ra=ra, dec=dec, unit=(u.degree, u.degree))
  #print(radec)
  # 2023-09-15
  #   Suddenly, objinfoFlag became objInfoFlag !!!
  #   It took several months to notice the fact.
  #   The keyword in the private sql database is still objinfoflag!
  columns= ["objID", "objInfoFlag", "qualityFlag", 
           "raMean", "decMean", "raMeanErr", "decMeanErr",
           "nStackDetections", "nDetections",
           "gQfPerfect", "gMeanPSFMag", "gMeanPSFMagErr", "gFlags",
           "rQfPerfect", "rMeanPSFMag", "rMeanPSFMagErr", "rFlags",
           "iQfPerfect", "iMeanPSFMag", "iMeanPSFMagErr", "iFlags",
           "zQfPerfect", "zMeanPSFMag", "zMeanPSFMagErr", "zFlags",
           "yQfPerfect", "yMeanPSFMag", "yMeanPSFMagErr", "yFlags"]
  dr = "dr2"
  tabletype = "mean"

  res = Catalogs.query_criteria(
      coordinates=radec, radius=radius, catalog="PANSTARRS", 
      gMeanPSFMag=[("lte", magmax),("gte", magmin)], 
      rMeanPSFMag=[("lte", magmax),("gte", magmin)],
      iMeanPSFMag=[("lte", magmax),("gte", magmin)],
      table=tabletype, data_release=dr) 
  res = res[columns]
  
  # Avoid int64 error for objID
  ID_str = [str(res[i]["objID"]) for i in range(len(res))]
  res["objID"] = ID_str

  df = res.to_pandas()
  df = df.fillna(999)
  return df

In [9]:
def log10err(val, err):
  """Calculate log10 error.

  Parameter
  ---------
  val : float
    value of the parameter
  err : float
    error of the parameter

  Return
  ------
  errlog10 : float 
    calculated log10 error
  """
  errlog10 = err/val/np.log(10)
  return errlog10


def check_objinfo(flag, n_checked=[2, 4, 6, 7, 8, 23, 24], verbose=False):
    """
    Check object info. with objinfoFlag.
    Return False if at least one content is flagged.
    
    
    Parameters
    ----------
    flag : int
        objinfoFlag
    n_flag : array-like
        flag value in decimal forms (from 0 to 31)
        By default, these flags are used.
            2 : CRF quasar,
            4 : possible  QSO
            6 : Possible RR Lyra
            7 : variable based on ChiSq
            8 : transient
            23 : Extended in PS
            24 : Extended in external data
    verbose : bool
        output messages

    Return
    ------
    True or False
    """
    # 31 ! not 32 (see header)
    N_bit = 31
    # Remove "0b"
    b_flag =  bin(int(flag))[2:]
    #print(f"flag : {flag}")
    #print(f"original length = {len(b_flag)}")
    #print(b_flag)
    # 0 padding
    b_flag = format(int(b_flag), f"031")
    #print(f"0 padded length = {len(b_flag)}")
    #print(b_flag)

    # PS_checked = [2,4,6,7,8,23,24]
    for n in range(N_bit):
        # From the last number
        # ex) n==0 (whether the last character==1 or not)
        #     
        if (b_flag[N_bit-(n+1)] == "1") & (n in n_checked):
            if verbose:
                print(f"    {n:02d} : {_info_flag['disc'][n]}")
            # Return False at least one content is flagged
            return False
    return True


def check_objfilter(flag, n_checked=[24], verbose=False):
    """
    Check object filter flag with e.g., gFlag.
    Return False if at least one content is flagged.
    
    Parameters
    ----------
    flag : int
        objinfoFlag
    n_flag : array-like
        flag value in decimal forms (from 0 to 24)
        By default, 24 is used.
            24 : Extended in this band
    verbose : bool
        output messages

    Return
    ------
    True or False
    """
    # 25 ! not 26 (see header)
    N_bit = 25
    # Remove "0b"
    b_flag =  bin(int(flag))[2:]
    #print(f"flag : {flag}")
    #print(f"original length = {len(b_flag)}")
    #print(b_flag)
    # 0 padding
    b_flag = format(int(b_flag), f"025")
    #print(f"0 padded length = {len(b_flag)}")
    #print(b_flag)

    for n in range(N_bit):
        # From the last number
        # ex) n==0 (whether the last character==1 or not)
        #     
        if (b_flag[N_bit-(n+1)] == "1") & (n in n_checked):
            if verbose:
                print(f"    {n:02d} : {_filter_flag['disc'][n]}")
            # Return False at least one content is flagged
            return False
    return True



In [10]:
# For asteroids
def make_cutout_sssb(target, fi_lists, center_lists, bands, rad_list=[8], key_jd="JD_MED", fov_arcsec=60, sig_min=0.5, sig_max=5, out="image.png"):
    """Make cutout of SSSBs.

    Parameters
    ----------
    target : str
        target name
    fi_lists : array-like   
        array of fits files 
    center_lists : array-like
        array of center coordinate in pix
    bands : array-like
        g, r, i or g, r, z
    rad_list : array-like
        list of aperture radius
    key_jd : str
        keyword for julian day
    cutout_arcsec : float
        size of cutout in arcsec
    sig_min : float
        minimum sigma of dynamic range
    sig_max : float
        maximum sigma of dynamic range
    out : str
        output filename
    """
    num_sets = len(fi_lists)
    num_bands = len(bands)


    fig, axes = plt.subplots(num_sets, num_bands, figsize=(4*num_bands, 4*num_sets))
    df_list = []

    for i in range(num_sets):
        print(f"Set {i+1}")
        f_list = fi_lists[i]
        center_list = center_lists[i]

        # Frame id (to remove data later)
        if bands == ["g", "r", "i"]:
            frameid = f"gri{i+1}"
        elif bands == ["g", "r", "z"]:
            frameid = f"grz{i+1}"

        for j, (band, fi, center) in enumerate(zip(bands, f_list, center_list)):
            ax = axes[i, j] if num_sets > 1 else axes[j]
            ax.axis("off")

            if band == "g":
                plt.rcParams["image.cmap"] = "Greens_r"
            elif band == "r":
                plt.rcParams["image.cmap"] = "Reds_r"
            elif band == "i":
                plt.rcParams["image.cmap"] = "PuRd_r"
            elif band == "z":
                plt.rcParams["image.cmap"] = "Purples_r"

            hdu = fits.open(fi)
            data = hdu[0].data
            hdr = hdu[0].header
            jd = hdr[key_jd]

            # Obtain gain value.
            gain = hdr["GAIN"]

            assert len(data.shape)==2, "Input 2-d fits!"
            ny, nx = data.shape
        
            # Background ===================================================
            data = data.byteswap().newbyteorder()

            bg_engine = sep.Background(data)
            bg_global = bg_engine.globalback
            # Used to estimate flux error
            bg_rms = bg_engine.globalrms

            # Just remove typical background
            data -= bg_global
            # Background ===================================================

            # Centroid =====================================================
            x0, y0 = center
            print(f"  {band}-band")
            print(f"    Original center (x0, y0) = ({x0:.1f}, {y0:.1f})")
            # Centroid 
            sigma = 2
            x1, y1, wflag = sep.winpos(data, x0, y0, sigma)
            print(f"    Winpos center (x1, y1) = ({x1:.1f}, {y1:.1f})")
            # Centroid =====================================================


            # Cut the data ========================================================
            # 2. by xcenter, ycenter, and width
            # When (xc, yc) = (51, 51) and width = 50,
            # cut region is in 1 <= x <= 101 and 1 <= y <= 101 in pixel.
            # Cut region is in 0 <= x <= 100 and 0 <= y <= 100 in python.
            # width should be even

            # Harf width in pixel (0.35 arcsec/pix)
            wi = int(fov_arcsec/0.35/2.)
            #xc, yc = center
            xmin, xmax = x1 - wi - 1, x1 + wi
            ymin, ymax = y1 - wi - 1, y1 + wi
            xmin, xmax = int(xmin), int(xmax)
            ymin, ymax = int(ymin), int(ymax)
            # Used to estimate local background
            data_cut = data[ymin:ymax, xmin:xmax]
            ny, nx = data_cut.shape
            print(f"Data length (Nx, Ny)=({nx}, {ny})")
            # Cut the data ========================================================

            data = np.ascontiguousarray(data)
            # Remove local background again
            bg_global_local = np.median(data_cut)
            data -= bg_global_local
            
            vmin, vmax = sig_min*bg_rms, sig_max*bg_rms
            # Plot 
            ax.imshow(data, vmin=vmin, vmax=vmax)

            # North is up, east is left
            ax.set_xlim([xmin, xmax])
            ax.set_ylim([ymin, ymax])


            # Do photometry 
            for rad in rad_list:
                flux, fluxerr, eflag = sep.sum_circle(data, [x1], [y1], r=rad, err=bg_rms, gain=gain)
                flux = flux[0]
                fluxerr = fluxerr[0]
                if j == 0:
                    flux_list = []
                    fluxerr_list = []
                flux_list.append(flux)
                fluxerr_list.append(fluxerr)
        
                # Draw circle
                ax.add_patch(Circle((x1, y1), rad, edgecolor='black', facecolor='none', linewidth=1, ls='solid'))

                # Make a dataframe
                df = pd.DataFrame({"target": [target], "objtype": ["ref"], "jd": [jd], "band": [band], "frameid": [frameid], "radius": [rad], "flux":[flux], "fluxerr": [fluxerr]})
                df_list.append(df)

            if j == 0:
                ax.text(0.05, 0.05, f"Set {i+1}", transform=ax.transAxes, color="white", size=16)
        
    
    plt.tight_layout()
    plt.savefig(out, dpi=300)
    plt.close()

    df = pd.concat(df_list)
    return df


In [11]:
# For stars
def make_cutout_star(target, fi_lists, ra, dec, bands, rad_list=[8], key_jd="JD_MED", fov_arcsec=60, sig_min=0.5, sig_max=5, out="image.png"):
    """Make cutout of stars.

    Parameters
    ----------
    target : str
        target name
    fi_lists : array-like   
        array of fits files 
    center_lists : array-like
        array of center coordinate in pix
    bands : array-like
        g, r, i or g, r, z
    rad_list : array-like
        list of aperture radius
    key_jd : str
        keyword for julian day
    cutout_arcsec : float
        size of cutout in arcsec
    sig_min : float
        minimum sigma of dynamic range
    sig_max : float
        maximum sigma of dynamic range
    out : str
        output filename
    """
    num_sets = len(fi_lists)
    num_bands = len(bands)

    fig, axes = plt.subplots(num_sets, num_bands, figsize=(4*num_bands, 4*num_sets))
    df_list = []

    for i in range(num_sets):
        f_list = fi_lists[i]

        # Frame id (to remove data later)
        if bands == ["g", "r", "i"]:
            frameid = f"gri{i+1}"
        elif bands == ["g", "r", "z"]:
            frameid = f"grz{i+1}"

        for j, (band, fi) in enumerate(zip(bands, f_list)):
            ax = axes[i, j] if num_sets > 1 else axes[j]
            ax.axis("off")

            if band == "g":
                plt.rcParams["image.cmap"] = "Greens_r"
            elif band == "r":
                plt.rcParams["image.cmap"] = "Reds_r"
            elif band == "i":
                plt.rcParams["image.cmap"] = "PuRd_r"
            elif band == "z":
                plt.rcParams["image.cmap"] = "Purples_r"

            hdu = fits.open(fi)
            data = hdu[0].data
            hdr = hdu[0].header
            w = wcs(header=hdr)
            jd = hdr[key_jd]
    
            # Obtain gain value.
            gain = hdr["GAIN"]

            assert len(data.shape)==2, "Input 2-d fits!"
            ny, nx = data.shape

            # Background ===================================================
            data = data.byteswap().newbyteorder()

            bg_engine = sep.Background(data)
            bg_global = bg_engine.globalback
            # Used to estimate flux error
            bg_rms = bg_engine.globalrms

            # Just remove typical background
            data -= bg_global
            # Background ===================================================


            # Centroid =====================================================
            # Get center coordinate from RA and Dec
            cra, cdec  = w.all_pix2world(1000, 500, 0)
            x_star, y_star  = w.all_world2pix(ra, dec, 0)
            print(f"  Original center (x, y) = ({x_star:.1f}, {y_star:.1f})")
    
            # Centroid 
            sigma = 5
            data = data.astype(data.dtype.newbyteorder('='))
            xwin, ywin, wflag = sep.winpos(data, x_star, y_star, sigma)
            center = [xwin, ywin]
            print(f"  Winpos center (x, y) = ({xwin:.1f}, {ywin:.1f})")
            # Centroid =====================================================


           # Cut the data ========================================================
            # 2. by xcenter, ycenter, and width
            # When (xc, yc) = (51, 51) and width = 50,
            # cut region is in 1 <= x <= 101 and 1 <= y <= 101 in pixel.
            # Cut region is in 0 <= x <= 100 and 0 <= y <= 100 in python.
            # width should be even

            # Harf width in pixel (0.35 arcsec/pix)
            wi = int(fov_arcsec/0.35/2.)
        
            xmin, xmax = xwin - wi - 1, xwin + wi
            ymin, ymax = ywin - wi - 1, ywin + wi
            xmin, xmax = int(xmin), int(xmax)
            ymin, ymax = int(ymin), int(ymax)
            data_cut = data[ymin:ymax, xmin:xmax]
            ny, nx = data_cut.shape
            print(f"Data length (Nx, Ny)=({nx}, {ny})")
            # Cut the data ========================================================

            # Remove local background again
            bg_global_local = np.median(data_cut)
            data -= bg_global_local

            vmin, vmax = sig_min*bg_rms, sig_max*bg_rms
            # Plot 
            ax.imshow(data, vmin=vmin, vmax=vmax)

            # North is up, east is left
            ax.set_xlim([xmin, xmax])
            ax.set_ylim([ymin, ymax])

            # Do photometry 
            for rad in rad_list:

                flux, fluxerr, eflag = sep.sum_circle(data, [xwin], [ywin], r=rad, err=bg_rms, gain=gain)
                flux = flux[0]
                fluxerr = fluxerr[0]
                if j == 0:
                    flux_list = []
                    fluxerr_list = []
                flux_list.append(flux)
                fluxerr_list.append(fluxerr)

                # Draw circle
                ax.add_patch(Circle((xwin, ywin), rad, edgecolor='black', facecolor='none', linewidth=1, ls='solid'))

                # Make a dataframe
                df = pd.DataFrame({"target": [target], "objtype": ["ref"], "jd": [jd], "band": [band], "frameid": [frameid], "radius": [rad], "flux":[flux], "fluxerr": [fluxerr]})
                df_list.append(df)
            

            # Plot band and flux
            boxdic = {
                "facecolor" : "black",
                "edgecolor" : "white",
                "boxstyle" : "Round",
                "linewidth" : 2
                }
            ax.text(0.05, 0.8, f"{band}: {flux:.1f}", transform=ax.transAxes, color="white", size=20, bbox=boxdic)
        
            if j == 0:
                ax.text(0.05, 0.05, f"Set {i+1}", transform=ax.transAxes, color="white", size=16)

    
    plt.tight_layout()
    plt.savefig(out, dpi=300)
    plt.close()

    df = pd.concat(df_list)
    return df


In [12]:
def make_ds9_region(ra_list, dec_list, out_region):
    """Make a region file for ds9

    Parameters
    ----------
    ra_list : array-like
        list of ra in deg
    dec_list : array-like
        list of dec in deg
    out_region : str
        output filename
    """
    header = [
        '# Region file format: DS9 version 4.1',
        'global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" '
        'select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1',
        'fk5'
    ]
    
    with open(out_region, 'w') as f:
        for line in header:
            f.write(line + '\n')
        
        for i, (ra, dec) in enumerate(zip(ra_list, dec_list), start=1):

            radius = '4.000"' 
            f.write(f'circle({ra},{dec},{radius}) # text={{star{i}}}\n')


def phot_PSstars(ra0_list, dec0_list, radec_rad, PSmagmin, PSmagmax, bands, out_region):
    """Select PS stars for photometry.

    Parameters
    ----------
    ra0_list : array-like
        original ra in deg
    dec0_list : array-like
        original dec in deg
    radec_rad : float
        search region in deg
    PSmagmin : float
        minimum magnitude to be considered
    PSmagmax : float
        maximum magnitude to be considered
    bands : array-like
        list of filters
    out_region : str
        output filename for ds9
    """
    ## Color threhold
    g_r_max = 1.1
    g_r_min = 0.0
    r_i_max = 0.8
    r_i_min = 0.0

    # Select stars by colors
    ra1_list, dec1_list = [], []
    for idx, (ra_star, dec_star) in enumerate(zip(ra0_list, dec0_list)):
        print(f"{idx+1}-th, (ra, dec) = ({ra_star}, {dec_star})")

        df_PS = query_ps(ra_star, dec_star, radius=radec_rad, magmin=PSmagmin, magmax=PSmagmax)

        if len(df_PS) == 0:
            print(f"  No stars found. (maybe ghost image)")
            print(f"  Do not use {idx+1}-th star\n")
            continue


        if len(df_PS) > 1:
            print(f"  More than 1 stars found. (could be contamination)")
            print(f"  Do not use {idx+1}-th star\n")
            continue
            
        # Check colors 
        g_PS, r_PS, i_PS = df_PS["gMeanPSFMag"][0], df_PS["rMeanPSFMag"][0], df_PS["iMeanPSFMag"][0]
        if g_PS - r_PS > g_r_max:
            print(f"  g-r is too red ({g_PS - r_PS:.1f})")
            print(f"  Do not use {idx+1}-th star\n")
            continue
        if g_PS - r_PS < g_r_min:
            print(f"  g-r is too blue ({g_PS - r_PS:.1f})")
            print(f"  Do not use {idx+1}-th star\n")
            continue
        if r_PS - i_PS > r_i_max:
            print(f"  r-i is too red ({r_PS - i_PS:.1f})")
            print(f"  Do not use {idx+1}-th star\n")
            continue
        if r_PS - i_PS < r_i_min:
            print(f"  r-i is too blue ({r_PS - i_PS:.1f})")
            print(f"  Do not use {idx+1}-th star\n")
            continue
        ra_cat, dec_cat = df_PS["raMean"][0], df_PS["decMean"][0]

       
        # Remove stars by PS objfilterflag (24. extended source)
        objflag_list = []
        for b in bands:
            # g.e., gFlags
            objflag = df_PS[f"{b}Flags"][0]
            objflag_list.append(objflag)
        if not check_objfilter(objflag_list[0]):
            print(f"  gFlags is {objflag}")
            print(f"  Do not use {idx+1}-th star\n")
            continue
        if not check_objfilter(objflag_list[1]):
            print(f"  rFlags is {objflag}")
            print(f"  Do not use {idx+1}-th star\n")
        if not check_objfilter(objflag_list[2]):
            print(f"  i/zFlags is {objflag}")
            print(f"  Do not use {idx+1}-th star\n")
        
        # Remove stars by PS objinfoFlag (many outliers......)
        objinfoflag = df_PS["objInfoFlag"][0]
        if not check_objinfo(objinfoflag):
            print(f"  objInfoFlag is {objinfoflag}")
            print(f"  Do not use {idx+1}-th star\n")
            continue

        print(f"  Use {idx+1}-th star\n")
        ra1_list.append(ra_cat)
        dec1_list.append(dec_cat)

    print()
    print(f"-> Selected N={len(ra1_list)}")
    ra_str  = ' '.join(str(num) for num in ra1_list)
    dec_str  = ' '.join(str(num) for num in dec1_list)
    print(f"--ra {ra_str} --dec {dec_str}")

    # Make region file
    make_ds9_region(ra1_list, dec1_list, out_region)
    return ra1_list, dec1_list

    


## 3I

In [13]:
# 2025-07-15 gri 7 sets
bands = ["g", "r", "i"]
center_lists = [
    [(1080, 643), (1066, 653), (1078, 633)],
    [(1082, 635), (1068, 645), (1079, 625)],
    [(1081, 635), (1067, 645), (1077, 625)],
    [(1078, 630), (1064, 640), (1075, 620)],
    [(1078, 637), (1064, 647), (1075, 627)],
    [(1082, 640), (1068, 648), (1079, 629)],
    [(1085, 630), (1070, 640), (1085, 620)],
]
fi_lists = []
for i in range(len(center_lists)):
    f_g = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/3I_gri/shift_600s_{stack}/nonsid_3I_gri_600s_0_00{i+1}.fits"
    f_r = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/3I_gri/shift_600s_{stack}/nonsid_3I_gri_600s_1_00{i+1}.fits"
    f_i = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/3I_gri/shift_600s_{stack}/nonsid_3I_gri_600s_2_00{i+1}.fits"
    fi_lists.append([f_g, f_r, f_i])

out = f"fig/3I_20250715_gri_{stack}_600s_7sets.jpg"
df_3I_gri = make_cutout_sssb("3I", fi_lists, center_lists, bands, rad_list=rad_list, fov_arcsec=fov_arcsec, sig_min=sig_min, sig_max=sig_max, out=out)


# 2025-07-15 g, r, z, 600 s, 5 set
bands = ["g", "r", "z"]
center_lists = [
    [(1080, 643), (1066, 653), (1078, 633)],
    [(1088, 643), (1074, 652), (1086, 632)],
    [(1085, 642), (1070, 652), (1082, 635)],
    [(1076, 633), (1062, 643), (1073, 623)],
    [(1076, 638), (1062, 647), (1073, 627)],
    ]

fi_lists = []
for i in range(len(center_lists)):
    f_g = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/3I_grz/shift_600s_{stack}/nonsid_3I_grz_600s_0_00{i+1}.fits"
    f_r = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/3I_grz/shift_600s_{stack}/nonsid_3I_grz_600s_1_00{i+1}.fits"
    f_z = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/3I_grz/shift_600s_{stack}/nonsid_3I_grz_600s_2_00{i+1}.fits"
    fi_lists.append([f_g, f_r, f_z])

out = f"fig/3I_20250715_grz_{stack}_600s_5sets.jpg"
df_3I_grz = make_cutout_sssb("3I", fi_lists, center_lists, bands, rad_list=rad_list, fov_arcsec=fov_arcsec, sig_min=sig_min, sig_max=sig_max, out=out)


df_3I = pd.concat([df_3I_gri, df_3I_grz])
df_3I = df_3I.reset_index(drop=True)

jd_min = np.min(df_3I["jd"])
df_3I["instmag"] = -2.5*np.log10(df_3I["flux"])
df_3I["instmagerr"] = 2.5*log10err(df_3I["flux"], df_3I["fluxerr"])
df_3I["elapsed_min"] = (df_3I["jd"] - jd_min)*24.*60.

df_3I.to_csv(f"{outdir}/photres_3I_{stack}_600s_20250715.txt", sep=" ")

Set 1
  g-band
    Original center (x0, y0) = (1080.0, 643.0)
    Winpos center (x1, y1) = (1079.8, 639.8)
Data length (Nx, Ny)=(85, 85)
  r-band
    Original center (x0, y0) = (1066.0, 653.0)
    Winpos center (x1, y1) = (1064.1, 649.6)
Data length (Nx, Ny)=(85, 85)
  i-band
    Original center (x0, y0) = (1078.0, 633.0)
    Winpos center (x1, y1) = (1075.0, 630.0)
Data length (Nx, Ny)=(85, 85)
Set 2
  g-band
    Original center (x0, y0) = (1082.0, 635.0)
    Winpos center (x1, y1) = (1082.4, 633.0)
Data length (Nx, Ny)=(85, 85)
  r-band
    Original center (x0, y0) = (1068.0, 645.0)
    Winpos center (x1, y1) = (1066.9, 642.4)
Data length (Nx, Ny)=(85, 85)
  i-band
    Original center (x0, y0) = (1079.0, 625.0)
    Winpos center (x1, y1) = (1078.2, 623.0)
Data length (Nx, Ny)=(85, 85)
Set 3
  g-band
    Original center (x0, y0) = (1081.0, 635.0)
    Winpos center (x1, y1) = (1079.7, 632.5)
Data length (Nx, Ny)=(85, 85)
  r-band
    Original center (x0, y0) = (1067.0, 645.0)
    Winpo

## Stars in 3I field

In [14]:
# Stars
bands = ["g", "r", "i"]
# RA and Dec in deg
ra1, dec1 = 263.7083, -18.3705
ra2, dec2 = 263.729, -18.377
ra3, dec3 = 263.7549, -18.3819
ra4, dec4 = 263.7072, -18.3776
ra5, dec5 = 263.689, -18.386
ra6, dec6 = 263.7025, -18.361
ra7, dec7 = 263.706, -18.3601
ra8, dec8 = 263.670, -18.348
ra9, dec9 = 263.6784, -18.336
ra10, dec10 = 263.680, -18.3499
ra11, dec11 = 263.6909, -18.3731
ra12, dec12 = 263.7285, -18.3905
ra13, dec13 = 263.7315, -18.3995
ra14, dec14 = 263.750, -18.378
ra15, dec15 = 263.740, -18.367
ra16, dec16 = 263.760, -18.3717
ra17, dec17 = 263.7676, -18.3762
ra18, dec18 = 263.7239, -18.3593
ra19, dec19 = 263.7131, -18.3547
ra20, dec20 = 263.7234, -18.3901

ra0_list =[
    ra1, ra2, ra3, ra4, ra5, 
    ra6, ra7, ra8, ra9, ra10, 
    ra11, ra12, ra13, ra14, ra15,
    ra16, ra17, ra18, ra19, ra20, 
    ]
dec0_list =[
    dec1, dec2, dec3, dec4, dec5, 
    dec6, dec7, dec8, dec9, dec10,
    dec11, dec12, dec13, dec14, dec15,
    dec16, dec17, dec18, dec19, dec20,
    ]

# 30 arcsec = 0.0083 deg
# 10 arcsec = 0.0027 deg
#  5 arcsec = 0.0014 deg
radec_rad = 0.0014
PSmagmin, PSmagmax = 11, 18

# Selection of stars
ds9_region = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/3I_gri/shift_600s_{stack}/ds9_PSstars.reg"
ra1_list, dec1_list = phot_PSstars(ra0_list, dec0_list, radec_rad, PSmagmin, PSmagmax, bands, ds9_region)


1-th, (ra, dec) = (263.7083, -18.3705)
  Use 1-th star

2-th, (ra, dec) = (263.729, -18.377)
  Use 2-th star

3-th, (ra, dec) = (263.7549, -18.3819)
  g-r is too red (1.1)
  Do not use 3-th star

4-th, (ra, dec) = (263.7072, -18.3776)
  Use 4-th star

5-th, (ra, dec) = (263.689, -18.386)
  g-r is too red (1.3)
  Do not use 5-th star

6-th, (ra, dec) = (263.7025, -18.361)
  gFlags is 17023288
  Do not use 6-th star

7-th, (ra, dec) = (263.706, -18.3601)
  Use 7-th star

8-th, (ra, dec) = (263.67, -18.348)
  Use 8-th star

9-th, (ra, dec) = (263.6784, -18.336)
  No stars found. (maybe ghost image)
  Do not use 9-th star

10-th, (ra, dec) = (263.68, -18.3499)


  Use 10-th star

11-th, (ra, dec) = (263.6909, -18.3731)
  Use 11-th star

12-th, (ra, dec) = (263.7285, -18.3905)
  Use 12-th star

13-th, (ra, dec) = (263.7315, -18.3995)
  Use 13-th star

14-th, (ra, dec) = (263.75, -18.378)
  Use 14-th star

15-th, (ra, dec) = (263.74, -18.367)
  Use 15-th star

16-th, (ra, dec) = (263.76, -18.3717)
  Use 16-th star

17-th, (ra, dec) = (263.7676, -18.3762)
  Use 17-th star

18-th, (ra, dec) = (263.7239, -18.3593)
  Use 18-th star

19-th, (ra, dec) = (263.7131, -18.3547)
  Use 19-th star

20-th, (ra, dec) = (263.7234, -18.3901)
  No stars found. (maybe ghost image)
  Do not use 20-th star


-> Selected N=15
--ra 263.7084515434115 263.7292535952521 263.70714324033054 263.7061392955667 263.6709135688206 263.6805755702292 263.6907522318014 263.72853377271775 263.7315546056891 263.750965871465 263.7404981353641 263.7601747169622 263.76774573509977 263.7240048690235 263.7131604299183 --dec -18.37060139682166 -18.377117659731585 -18.377640380248177 -18.3

In [15]:
# 2025-07-15
# Save all results and concatnate later
df_star_list = []

for idx, (ra_star, dec_star) in enumerate(zip(ra1_list, dec1_list)):

    df_PS = query_ps(ra_star, dec_star, radius=radec_rad, magmin=PSmagmin, magmax=PSmagmax)
    assert len(df_PS)==1, df_PS
     
    g_PS, r_PS, i_PS = df_PS["gMeanPSFMag"][0], df_PS["rMeanPSFMag"][0], df_PS["iMeanPSFMag"][0]

    ra_cat, dec_cat = df_PS["raMean"][0], df_PS["decMean"][0]
    objid = df_PS["objID"][0]

    # gri 7 sets
    bands = ["g", "r", "i"]
    fi_lists = []
    for i in range(7):
        f_g = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/3I_gri/shift_600s_{stack}/sid_3I_gri_600s_0_00{i+1}.fits"
        f_r = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/3I_gri/shift_600s_{stack}/sid_3I_gri_600s_1_00{i+1}.fits"
        f_i = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/3I_gri/shift_600s_{stack}/sid_3I_gri_600s_2_00{i+1}.fits"
        fi_lists.append([f_g, f_r, f_i])


    out = f"fig/PSstar{idx+1:02d}_20250715_gri_{stack}_600s_7sets.jpg"
    df_star = make_cutout_star(objid, fi_lists, ra_cat, dec_cat, bands, rad_list=rad_list, fov_arcsec=fov_arcsec, sig_min=sig_min, sig_max=sig_max, out=out)

    # Add catalog mag
    df_star["PSmag"] = 0
    for band in bands:
        PSmag_band = df_PS[f"{band}MeanPSFMag"][0]
        df_star.loc[df_star["band"] == band, "PSmag"] = PSmag_band

    df_star_list.append(df_star)

    # grz 5 sets
    bands = ["g", "r", "z"]
    fi_lists = []
    for i in range(5):
        f_g = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/3I_grz/shift_600s_mean/sid_3I_grz_600s_0_00{i+1}.fits"
        f_r = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/3I_grz/shift_600s_mean/sid_3I_grz_600s_1_00{i+1}.fits"
        f_z = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/3I_grz/shift_600s_mean/sid_3I_grz_600s_2_00{i+1}.fits"
        fi_lists.append([f_g, f_r, f_z])

    out = f"fig/PSstar{idx+1:02d}_20250715_grz_{stack}_600s_5sets.jpg"
    df_star = make_cutout_star(objid, fi_lists, ra_cat, dec_cat, bands, rad_list=rad_list, fov_arcsec=fov_arcsec, sig_min=sig_min, sig_max=sig_max, out=out)

    # Add catalog mag
    df_star["PSmag"] = 0
    for band in bands:
        PSmag_band = df_PS[f"{band}MeanPSFMag"][0]
        df_star.loc[df_star["band"] == band, "PSmag"] = PSmag_band
        
    df_star_list.append(df_star)


# Check photometry of stars
df_ref = pd.concat(df_star_list)
jd_min = np.min(df_ref["jd"])
df_ref["instmag"] = -2.5*np.log10(df_ref["flux"])
df_ref["instmagerr"] = 2.5*log10err(df_ref["flux"], df_ref["fluxerr"])
df_ref["elapsed_min"] = (df_ref["jd"] - jd_min)*24.*60.

df_ref.to_csv(f"{outdir}/photres_ref_{stack}_600s_20250715.txt", sep=" ")


Set DATE-END to '2025-07-15T12:42:50.901' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T12:42:50.599' from MJD-END'. [astropy.wcs.wcs]


  Original center (x, y) = (1423.0, 740.3)
  Winpos center (x, y) = (1422.1, 741.0)
Data length (Nx, Ny)=(85, 85)
  Original center (x, y) = (1407.1, 751.6)
  Winpos center (x, y) = (1406.1, 752.4)
Data length (Nx, Ny)=(85, 85)
  Original center (x, y) = (1418.4, 731.9)
  Winpos center (x, y) = (1417.7, 731.9)
Data length (Nx, Ny)=(85, 85)


Set DATE-END to '2025-07-15T13:15:06.097' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T13:15:05.700' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T13:15:05.898' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T13:47:58.704' from MJD-END'. [astropy.wcs.wcs]


  Original center (x, y) = (1289.0, 727.0)
  Winpos center (x, y) = (1289.4, 727.5)
Data length (Nx, Ny)=(85, 85)
  Original center (x, y) = (1273.4, 737.8)
  Winpos center (x, y) = (1273.7, 738.2)
Data length (Nx, Ny)=(85, 85)
  Original center (x, y) = (1284.8, 717.7)
  Winpos center (x, y) = (1285.2, 718.2)
Data length (Nx, Ny)=(85, 85)
  Original center (x, y) = (1151.2, 717.2)
  Winpos center (x, y) = (1148.9, 720.2)
Data length (Nx, Ny)=(85, 85)


Set DATE-END to '2025-07-15T13:47:58.298' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T13:47:58.497' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T14:38:19.196' from MJD-END'. [astropy.wcs.wcs]


...

Set DATE-END to '2025-07-15T14:00:17.700' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T14:14:10.596' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T14:14:10.199' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T14:14:10.398' from MJD-END'. [astropy.wcs.wcs]


  Original center (x, y) = (1053.0, 878.2)
  Winpos center (x, y) = (1052.9, 877.9)
Data length (Nx, Ny)=(85, 85)
  Original center (x, y) = (990.8, 875.2)
  Winpos center (x, y) = (989.7, 875.1)
Data length (Nx, Ny)=(85, 85)
  Original center (x, y) = (973.7, 884.4)
  Winpos center (x, y) = (973.9, 884.5)
Data length (Nx, Ny)=(85, 85)
  Original center (x, y) = (985.7, 865.2)
  Winpos center (x, y) = (985.9, 865.2)
Data length (Nx, Ny)=(85, 85)


Set DATE-END to '2025-07-15T14:24:13.504' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T14:24:13.098' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T14:24:13.297' from MJD-END'. [astropy.wcs.wcs]


  Original center (x, y) = (948.0, 877.2)
  Winpos center (x, y) = (947.3, 877.3)
Data length (Nx, Ny)=(85, 85)
  Original center (x, y) = (932.3, 886.1)
  Winpos center (x, y) = (931.8, 886.2)
Data length (Nx, Ny)=(85, 85)
  Original center (x, y) = (943.2, 867.0)
  Winpos center (x, y) = (943.4, 867.2)
Data length (Nx, Ny)=(85, 85)


/var/folders/xv/z4qp4sf50z10cwc8tgxysssw0000gn/T/ipykernel_99592/108843714.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '17.1968994140625' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_star.loc[df_star["band"] == band, "PSmag"] = PSmag_band


## Popescu (120 s)

In [16]:
# 2025-07-15 gri 2 set
bands = ["g", "r", "i"]
center_lists = [
    [(1102, 650), (1086, 660), (1097, 641)],
    [(1103, 650), (1088, 659), (1098, 641)],
    [(1101, 652), (1085, 662), (1097, 642)],
    [(1101, 652), (1085, 662), (1097, 642)],
]

fi_lists = []
for i in range(2):
    f_g = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/Popescu_gri/shift_120s_{stack}/nonsid_Popescu_gri_120s_0_00{i+1}.fits"
    f_r = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/Popescu_gri/shift_120s_{stack}/nonsid_Popescu_gri_120s_1_00{i+1}.fits"
    f_i = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/Popescu_gri/shift_120s_{stack}/nonsid_Popescu_gri_120s_2_00{i+1}.fits"
    fi_lists.append([f_g, f_r, f_i])

out = f"fig/Popsecu_20250715_gri_{stack}_120s_2sets.jpg"
df_P_gri = make_cutout_sssb("Popescu", fi_lists, center_lists, bands, rad_list=rad_list, fov_arcsec=fov_arcsec, out=out)


# 2025-07-15 grz 2 set
bands = ["g", "r", "z"]
center_lists = [
    [(1099, 650), (1085, 660), (1096, 641)],
    [(1100, 649), (1087, 659), (1097, 640)],
    [(1099, 651), (1085, 660), (1096, 640)],
    [(1099, 651), (1085, 661), (1096, 640)],
]

fi_lists = []
for i in range(2):
    f_g = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/Popescu_grz/shift_120s_{stack}/nonsid_Popescu_grz_120s_0_00{i+1}.fits"
    f_r = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/Popescu_grz/shift_120s_{stack}/nonsid_Popescu_grz_120s_1_00{i+1}.fits"
    f_z = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/Popescu_grz/shift_120s_{stack}/nonsid_Popescu_grz_120s_2_00{i+1}.fits"
    fi_lists.append([f_g, f_r, f_z])

out = f"fig/Popsecu_20250715_grz_{stack}_120s_2sets.jpg"
df_P_grz = make_cutout_sssb("Popescu", fi_lists, center_lists, bands, rad_list=rad_list, fov_arcsec=fov_arcsec, out=out)


df_P = pd.concat([df_P_gri, df_P_grz])
df_P = df_P.reset_index(drop=True)

jd_min = np.min(df_P["jd"])
df_P["instmag"] = -2.5*np.log10(df_P["flux"])
df_P["instmagerr"] = 2.5*log10err(df_P["flux"], df_P["fluxerr"])
df_P["elapsed_min"] = (df_P["jd"] - jd_min)*24.*60.

df_P.to_csv(f"{outdir}/photres_Popescu_{stack}_120s_20250715.txt", sep=" ")

Set 1
  g-band
    Original center (x0, y0) = (1102.0, 650.0)
    Winpos center (x1, y1) = (1100.9, 649.1)
Data length (Nx, Ny)=(85, 85)
  r-band
    Original center (x0, y0) = (1086.0, 660.0)
    Winpos center (x1, y1) = (1085.5, 658.9)
Data length (Nx, Ny)=(85, 85)
  i-band
    Original center (x0, y0) = (1097.0, 641.0)
    Winpos center (x1, y1) = (1096.6, 639.3)
Data length (Nx, Ny)=(85, 85)
Set 2
  g-band
    Original center (x0, y0) = (1103.0, 650.0)
    Winpos center (x1, y1) = (1100.3, 651.2)
Data length (Nx, Ny)=(85, 85)
  r-band
    Original center (x0, y0) = (1088.0, 659.0)
    Winpos center (x1, y1) = (1085.0, 660.9)
Data length (Nx, Ny)=(85, 85)
  i-band
    Original center (x0, y0) = (1098.0, 641.0)
    Winpos center (x1, y1) = (1096.3, 641.5)
Data length (Nx, Ny)=(85, 85)
Set 1
  g-band
    Original center (x0, y0) = (1099.0, 650.0)
    Winpos center (x1, y1) = (1098.3, 648.7)
Data length (Nx, Ny)=(85, 85)
  r-band
    Original center (x0, y0) = (1085.0, 660.0)
    Winpo

## Stars in Popescu field (120 s)

In [17]:
# Stars
# Avoid red stars (check i and z bands)
bands = ["g", "r", "i"]
# RA and Dec in deg
ra1, dec1 = 263.6540, -17.1456
ra2, dec2 = 263.625, -17.1557
ra3, dec3 = 263.6564, -17.1594
ra4, dec4 = 263.666, -17.1547
ra5, dec5 = 263.6117, -17.1589
ra6, dec6 = 263.6241, -17.1459
ra7, dec7 = 263.6121, -17.1347
ra8, dec8 = 263.6117, -17.1248
ra9, dec9 = 263.6035, -17.1461
ra10, dec10 = 263.5742, -17.1272

ra11, dec11 = 263.5775, -17.1364
ra12, dec12 = 263.5886, -17.1563
ra13, dec13 = 263.6019, -17.1360
ra14, dec14 = 263.6274, -17.1658
ra15, dec15 = 263.6509, -17.1497

ra16, dec16 = 263.6406, -17.141557
ra17, dec17 = 263.5997, -17.1203
ra18, dec18 = 263.6693, -17.1264
ra19, dec19 = 263.6944, -17.1353
ra20, dec20 = 263.6679, -17.1688

ra21, dec21 = 263.6407, -17.1678
ra22, dec22 = 263.6230, -17.1748
ra23, dec23 = 263.6427, -17.1754
ra24, dec24 = 263.5962, -17.1755
ra25, dec25 = 263.6802, -17.1806

ra0_list =[
    ra1, ra2, ra3, ra4, ra5, 
    ra6, ra7, ra8, ra9, ra10, 
    ra11, ra12, ra13, ra14, ra15, 
    ra16, ra17, ra18, ra19, ra20, 
    ra21, ra22, ra23, ra24, ra25, 
    ]
dec0_list =[
    dec1, dec2, dec3, dec4, dec5, 
    dec6, dec7, dec8, dec9, dec10, 
    dec11, dec12, dec13, dec14, dec15, 
    dec16, dec17, dec18, dec19, dec20, 
    dec21, dec22, dec23, dec24, dec25, 
    ]

# 30 arcsec = 0.0083 deg
# 10 arcsec = 0.0027 deg
#  5 arcsec = 0.0014 deg
radec_rad = 0.0014
PSmagmin, PSmagmax = 11, 18


# Selection of stars
ds9_region = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/Popescu_gri/shift_120s_{stack}/ds9_PSstars.reg"
ra1_list, dec1_list = phot_PSstars(ra0_list, dec0_list, radec_rad, PSmagmin, PSmagmax, bands, out_region=ds9_region)


1-th, (ra, dec) = (263.654, -17.1456)
  g-r is too red (1.3)
  Do not use 1-th star

2-th, (ra, dec) = (263.625, -17.1557)
  Use 2-th star

3-th, (ra, dec) = (263.6564, -17.1594)
  g-r is too red (1.1)
  Do not use 3-th star

4-th, (ra, dec) = (263.666, -17.1547)
  Use 4-th star

5-th, (ra, dec) = (263.6117, -17.1589)
  Use 5-th star

6-th, (ra, dec) = (263.6241, -17.1459)
  Use 6-th star

7-th, (ra, dec) = (263.6121, -17.1347)
  g-r is too red (1.4)
  Do not use 7-th star

8-th, (ra, dec) = (263.6117, -17.1248)
  Use 8-th star

9-th, (ra, dec) = (263.6035, -17.1461)
  Use 9-th star

10-th, (ra, dec) = (263.5742, -17.1272)
  Use 10-th star

11-th, (ra, dec) = (263.5775, -17.1364)
  Use 11-th star

12-th, (ra, dec) = (263.5886, -17.1563)
  g-r is too red (1.3)
  Do not use 12-th star

13-th, (ra, dec) = (263.6019, -17.136)
  g-r is too red (1.3)
  Do not use 13-th star

14-th, (ra, dec) = (263.6274, -17.1658)
  g-r is too red (1.4)
  Do not use 14-th star

15-th, (ra, dec) = (263.6509, 

  Use 17-th star

18-th, (ra, dec) = (263.6693, -17.1264)
  Use 18-th star

19-th, (ra, dec) = (263.6944, -17.1353)
  Use 19-th star

20-th, (ra, dec) = (263.6679, -17.1688)
  Use 20-th star

21-th, (ra, dec) = (263.6407, -17.1678)
  g-r is too red (1.1)
  Do not use 21-th star

22-th, (ra, dec) = (263.623, -17.1748)
  Use 22-th star

23-th, (ra, dec) = (263.6427, -17.1754)
  g-r is too red (1.2)
  Do not use 23-th star

24-th, (ra, dec) = (263.5962, -17.1755)
  g-r is too red (1.2)
  Do not use 24-th star

25-th, (ra, dec) = (263.6802, -17.1806)
  i/zFlags is 16777496
  Do not use 25-th star

  Use 25-th star


-> Selected N=14
--ra 263.6248806102392 263.6663239404475 263.611564934402 263.624055023923 263.611610216147 263.6034272620103 263.57422114796594 263.57750134897134 263.5996181284078 263.66944117676593 263.6945731543876 263.6680872122373 263.62294264338243 263.6802939305236 --dec -17.15573235082032 -17.15467184573923 -17.158812135454838 -17.145839555135403 -17.124607110548137 -

In [18]:
# 2025-07-15
# Save all results and concatnate later
df_star_list = []

for idx, (ra_star, dec_star) in enumerate(zip(ra1_list, dec1_list)):

    df_PS = query_ps(ra_star, dec_star, radius=radec_rad, magmin=PSmagmin, magmax=PSmagmax)
    assert len(df_PS)==1, df_PS
     
    g_PS, r_PS, i_PS = df_PS["gMeanPSFMag"][0], df_PS["rMeanPSFMag"][0], df_PS["iMeanPSFMag"][0]

    ra_cat, dec_cat = df_PS["raMean"][0], df_PS["decMean"][0]
    objid = df_PS["objID"][0]

    # gri 2 sets
    bands = ["g", "r", "i"]
    fi_lists = []
    for i in range(2):
        f_g = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/Popescu_gri/shift_120s_{stack}/sid_Popescu_gri_120s_0_00{i+1}.fits"
        f_r = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/Popescu_gri/shift_120s_{stack}/sid_Popescu_gri_120s_1_00{i+1}.fits"
        f_i = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/Popescu_gri/shift_120s_{stack}/sid_Popescu_gri_120s_2_00{i+1}.fits" 
        fi_lists.append([f_g, f_r, f_i])


    out = f"fig/PSstar{idx+1:02d}_20250715_{stack}_gri_120s_2sets.jpg"
    df_star = make_cutout_star(objid, fi_lists, ra_cat, dec_cat, bands, rad_list=rad_list, fov_arcsec=fov_arcsec, out=out)

    # Add catalog mag
    df_star["PSmag"] = 0
    for band in bands:
        PSmag_band = df_PS[f"{band}MeanPSFMag"][0]
        df_star.loc[df_star["band"] == band, "PSmag"] = PSmag_band

    df_star_list.append(df_star)

    # grz 2 sets
    bands = ["g", "r", "z"]
    fi_lists = []
    for i in range(2):
        f_g = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/Popescu_grz/shift_120s_{stack}/sid_Popescu_grz_120s_0_00{i+1}.fits"
        f_r = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/Popescu_grz/shift_120s_{stack}/sid_Popescu_grz_120s_1_00{i+1}.fits"
        f_z = f"/Users/beniyama/research/Seimei_3I/Seimei202507_3I/20250715/Popescu_grz/shift_120s_{stack}/sid_Popescu_grz_120s_2_00{i+1}.fits" 
        fi_lists.append([f_g, f_r, f_z])

    out = f"fig/PSstar{idx+1:02d}_20250715_grz_{stack}_120s_2sets.jpg"
    df_star = make_cutout_star(objid, fi_lists, ra_cat, dec_cat, bands, rad_list=rad_list, fov_arcsec=fov_arcsec, out=out)

    # Add catalog mag
    df_star["PSmag"] = 0
    for band in bands:
        PSmag_band = df_PS[f"{band}MeanPSFMag"][0]
        df_star.loc[df_star["band"] == band, "PSmag"] = PSmag_band
        
    df_star_list.append(df_star)


# Check photometry of stars
df_ref = pd.concat(df_star_list)
jd_min = np.min(df_ref["jd"])
df_ref["instmag"] = -2.5*np.log10(df_ref["flux"])
df_ref["instmagerr"] = 2.5*log10err(df_ref["flux"], df_ref["fluxerr"])
df_ref["elapsed_min"] = (df_ref["jd"] - jd_min)*24.*60.

df_ref.to_csv(f"{outdir}/photres_ref_{stack}_120s_20250715.txt", sep=" ")


Set DATE-END to '2025-07-15T13:02:05.404' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T13:02:04.998' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T13:02:05.196' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T13:34:25.602' from MJD-END'. [astropy.wcs.wcs]


  Original center (x, y) = (1256.4, 647.2)
  Winpos center (x, y) = (1255.2, 647.3)
Data length (Nx, Ny)=(85, 85)
  Original center (x, y) = (1241.0, 658.0)
  Winpos center (x, y) = (1239.9, 657.8)
Data length (Nx, Ny)=(85, 85)
  Original center (x, y) = (1252.0, 637.9)
  Winpos center (x, y) = (1251.2, 637.8)
Data length (Nx, Ny)=(85, 85)
  Original center (x, y) = (1223.7, 655.6)
  Winpos center (x, y) = (1222.2, 655.8)
Data length (Nx, Ny)=(85, 85)
  Original center (x, y) = (1208.4, 666.1)
  Winpos center (x, y) = (1207.1, 666.1)
Data length (Nx, Ny)=(85, 85)
  Original center (x, y) = (1219.5, 646.2)
  Winpos center (x, y) = (1218.6, 646.3)
Data length (Nx, Ny)=(85, 85)


Set DATE-END to '2025-07-15T13:34:25.101' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T13:34:25.404' from MJD-END'. [astropy.wcs.wcs]
/var/folders/xv/z4qp4sf50z10cwc8tgxysssw0000gn/T/ipykernel_99592/1169616324.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '17.135499954223633' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_star.loc[df_star["band"] == band, "PSmag"] = PSmag_band
Set DATE-END to '2025-07-15T12:59:13.001' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T12:59:12.500' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T12:59:12.803' from MJD-END'. [astropy.wcs.wcs]


  Original center (x, y) = (1256.4, 646.7)
  Winpos center (x, y) = (1255.4, 646.3)
Data length (Nx, Ny)=(85, 85)
  Original center (x, y) = (1241.0, 657.4)
  Winpos center (x, y) = (1240.1, 656.9)
Data length (Nx, Ny)=(85, 85)
  Original center (x, y) = (1251.3, 636.8)
  Winpos center (x, y) = (1251.5, 636.4)
Data length (Nx, Ny)=(85, 85)


Set DATE-END to '2025-07-15T13:31:27.204' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T13:31:26.797' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T13:31:26.996' from MJD-END'. [astropy.wcs.wcs]


...

/var/folders/xv/z4qp4sf50z10cwc8tgxysssw0000gn/T/ipykernel_99592/1169616324.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '17.44379997253418' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_star.loc[df_star["band"] == band, "PSmag"] = PSmag_band
Set DATE-END to '2025-07-15T12:59:13.001' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T12:59:12.500' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T12:59:12.803' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T13:31:27.204' from MJD-END'. [astropy.wcs.wcs]


  Original center (x, y) = (711.8, 394.7)
  Winpos center (x, y) = (712.8, 395.4)
Data length (Nx, Ny)=(85, 85)
  Original center (x, y) = (697.6, 402.4)
  Winpos center (x, y) = (698.6, 402.9)
Data length (Nx, Ny)=(85, 85)
  Original center (x, y) = (706.4, 383.0)
  Winpos center (x, y) = (708.4, 383.8)
Data length (Nx, Ny)=(85, 85)
  Original center (x, y) = (680.4, 399.5)
  Winpos center (x, y) = (680.9, 400.4)
Data length (Nx, Ny)=(85, 85)
  Original center (x, y) = (666.3, 407.1)
  Winpos center (x, y) = (666.7, 408.2)
Data length (Nx, Ny)=(85, 85)


Set DATE-END to '2025-07-15T13:31:26.797' from MJD-END'. [astropy.wcs.wcs]
Set DATE-END to '2025-07-15T13:31:26.996' from MJD-END'. [astropy.wcs.wcs]


  Original center (x, y) = (676.3, 388.3)
  Winpos center (x, y) = (676.7, 389.2)
Data length (Nx, Ny)=(85, 85)


/var/folders/xv/z4qp4sf50z10cwc8tgxysssw0000gn/T/ipykernel_99592/1169616324.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '17.44379997253418' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_star.loc[df_star["band"] == band, "PSmag"] = PSmag_band
